In [1]:
import sys
sys.path.append("../code")
import pandas as pd
import random
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import blotto_utils as bu
from agent import Agent
import agent_utils as au

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [5]:
pd.options.display.max_colwidth = 10

# 538 Data

In [6]:
weights = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

data_dir = '/Users/Alex/Documents/Alex/Yale/Fall 2019/CPSC_490/blotto/data/'

fivethirtyeight_data_dir = data_dir + 'fivethirtyeight/'

sorted(glob(fivethirtyeight_data_dir + '*'))

temp_df = pd.read_csv(glob(fivethirtyeight_data_dir + '*')[0])

new_column_names = list(map((lambda x : '_'.join(x.split(' ')).lower()), list(temp_df.columns)))
new_column_names[-1] = 'reason'

temp_df.rename(dict(zip(list(temp_df.columns), new_column_names)), axis=1, inplace=True)

temp_df['total_troops'] = temp_df['castle_1'] + \
                          temp_df['castle_2'] + \
                          temp_df['castle_3'] + \
                          temp_df['castle_4'] + \
                          temp_df['castle_5'] + \
                          temp_df['castle_6'] + \
                          temp_df['castle_7'] + \
                          temp_df['castle_8'] + \
                          temp_df['castle_9'] + \
                          temp_df['castle_10']

full_strats_df = temp_df[temp_df.total_troops == 100].copy()

full_strats_list = full_strats_df.to_dict('records')

simp_strats_list = [[int(x[1]) for x in list(y.items())[:10]] for y in full_strats_list]

## Divide into two players

In [7]:
random.seed(0)

In [8]:
a_strat_init = random.sample(simp_strats_list, 100)
b_strat_init = random.sample(simp_strats_list, 100)

In [9]:
len(a_strat_init), len(b_strat_init)

(100, 100)

# Agent

In [66]:
random.seed(0)

In [67]:
temp_noise = bu.generate_noise_arrays(1)

In [68]:
def evolve(a_strats, b_strats, generations, noise):
    for _ in range(generations):
        a_results, b_results = bu.eval_strats(a_strats, b_strats, weights)

        a_fittest = sorted(a_results, key=(lambda x: (x[1], -(x[0][-1]))), reverse=True)
        b_fittest = sorted(b_results, key=(lambda x: (x[1], -(x[0][-1]))), reverse=True)

        a_next_gen = []
        b_next_gen = []
        for i in range(14):
            for j in range(i+1, 15):
                a_next_gen.append(bu.mutate_noise(bu.make_integer(bu.cross([a_fittest[i][0], a_fittest[j][0]]), seed=(i+j+i*j)), temp_noise, seed=(i+j)))
                b_next_gen.append(bu.mutate_noise(bu.make_integer(bu.cross([b_fittest[i][0], b_fittest[j][0]]), seed=(i+j+i*j)), temp_noise, seed=(i+j)))

        a_strats = a_next_gen
        b_strats = b_next_gen
        
    return a_fittest, b_fittest, a_strats, b_strats

In [69]:
a_strat_init = random.sample(simp_strats_list, 100)
b_strat_init = random.sample(simp_strats_list, 100)

a_strats = a_strat_init
b_strats = b_strat_init

In [70]:
temp_a_strats, temp_b_strats = a_strats, b_strats

In [71]:
best_a, best_b = [], []

In [72]:
for i in range(200):
#     print(i)
    temp_a_fittest, temp_b_fittest, temp_a_strats, temp_b_strats = evolve(temp_a_strats, temp_b_strats, 1, temp_noise)
    best_a.append(temp_a_fittest[0][0])
    best_b.append(temp_b_fittest[0][0])
#     print('\n'.join(map(str,temp_a_fittest[:5])))

In [73]:
len(best_a), len(best_b)

(200, 200)

# Testing

In [86]:
for index in [10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200]:
    temp_a = Agent(best_a[:index])
    temp_b = Agent(best_b[:index])

    temp_sum = 0
    times = 1
    for _ in range(times):
        temp_sum += au.play_game(temp_a, temp_b, weights, weights, trials=100)
        print(type(temp_a))
    print(index, temp_sum/times)

UnboundLocalError: local variable 'b_strats' referenced before assignment

In [78]:
for index in [1, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200]:
    agent_a = Agent(best_a[:index])
    print(index, au.play_game(agent_a, simp_strats_list, weights, weights, trials=len(simp_strats_list)) / len(simp_strats_list))

1 0.760797342192691
10 0.646733111849391
15 0.6223698781838317
20 0.5825027685492802
25 0.5138427464008859
30 0.45071982281284606
40 0.3842746400885936
50 0.3610188261351052
60 0.35658914728682173
70 0.3289036544850498
80 0.3233665559246955
90 0.2978959025470653
100 0.27464008859357697
150 0.26245847176079734
200 0.3001107419712071


In [80]:
for index in [1, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200]:
    agent_b = Agent(best_b[:index])
    print(index, au.play_game(agent_b, simp_strats_list, weights, weights, trials=len(simp_strats_list)) / len(simp_strats_list))

1 0.7098560354374308
10 0.5116279069767442
15 0.49169435215946844
20 0.4418604651162791
25 0.416389811738649
30 0.3842746400885936
40 0.27464008859357697
50 0.292358803986711
60 0.26578073089701
70 0.25249169435215946
80 0.2702104097452935
90 0.21373200442967885
100 0.2070874861572536
150 0.1716500553709856
200 0.22591362126245848


In [85]:
type(agent_b)

agent.Agent

In [81]:
best_b[0]

[0, 1, 8, 14, 16, 1, 22, 3, 25, 10]

In [82]:
best_a[0]

[1, 1, 1, 15, 20, 1, 1, 1, 30, 29]

# ew

In [83]:
temp_a, temp_b = bu.eval_strats([[15,10,10,10,10,10,10,10,10,5]], simp_strats_list, weights)

In [84]:
sorted(temp_a, key=(lambda x: (x[1], -(x[0][-1]))), reverse=True)

[([15, 10, 10, 10, 10, 10, 10, 10, 10, 5], 391)]